In [ ]:
# create artificial dataset

import pandas as pd

data = {'measurementTimestamp': [1000000000000,
                                 1000000000001,
                                 1000000000002,
                                 1000000000003,
                                 1000000000004],
       'sourceMac': ['2d1ff804-c499-4163-b90f-003df1e4ec65',
                     '85a84e08-c967-41d0-b56b-522574796776',
                     '2c019c2d-7e9e-42b1-b3bd-1797c918d4a7',
                     '2c019c2d-7e9e-42b1-b3bd-1797c918d4a8',
                     'e1deda99-163f-4b68-9ade-e1f05d070bf2'],
       'coordinates': [[20,20],
                       [-20,40],
                       [-50,50],
                       [70,-20],
                       [40,30]],
       'x_error': [20,20,20,20,20],
       'y_error': [20,20,1,1.3,20]}
        
frame = pd.DataFrame(data)

In [16]:
# create artificial dataset

import pandas as pd

data = {'measurementTimestamp': [1000000000001],
       'sourceMac': ['2d1ff804-c499-4163-b90f-003df1e4ec65'],
       'coordinates': [[80,50]],
       'x_error': [50],
       'y_error': [50]}
        
frame = pd.DataFrame(data)

In [21]:
# real dataset

import json

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [22]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [23]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
chi2PerDof.columns = ['chi2PerDof']
probChi2 = pd.DataFrame(list(trackee['probChi2']))
probChi2.columns = ['probChi2']
nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
nMeasurements.columns = ['nMeasurements']

In [24]:
# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
frame = frame.join(chi2PerDof)
frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(nMeasurements)
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [25]:
frame = frame.sort_values(by='measurementTimestamp')

In [26]:
frame[:3]

,measurementTimestamp,processingTimestamp,sourceMac,coordinates,chi2PerDof,probChi2,x_error,y_error,nMeasurements
73,1436047367297,1436047381262,62e72aeb-2c41-44ed-88a6-3423267c1cb5,"[-7.459665, -75.723003, 0.0]",0.101141,0.750465,3.838867,0.298622,4
141,1436047367330,1436047381441,7c824b77-a38b-4c3d-bb6a-3e40f5373d97,"[46.207027, -29.514564, 0.0]",1.323366,0.249989,3.778767,2.691939,4
149,1436047367683,1436047381455,4ff97883-b9e8-4e2e-8b22-9bd093cfa401,"[-110.189892, -2.635529, 0.0]",3.053348,0.080571,5.761471,5.406795,4


In [7]:
def movingWindow(k):
    start = min(frame['measurementTimestamp']) + k * timestep
    stop = start + interval

    window = frame[(frame['measurementTimestamp'] >= start) & 
                       (frame['measurementTimestamp'] < stop)]

    return window

In [8]:
def createHistograms(window):
    grids = np.zeros((len(set(window['sourceMac'])), height,width))

    # dictionary of histograms (with mac addresses as keys)
    histos = dict(zip(set(window['sourceMac']), grids))
    
    # dictionary of normalization counts 
    norm_counts = dict( zip(set(window['sourceMac']), np.zeros(len(set(window['sourceMac'])))))
    
    return histos, norm_counts

In [9]:
def kernel(x, sigma): 
    return 1/(sqrt(2*pi) * sigma) * exp(-(x**2)/(2*sigma**2))

In [10]:
def smoothHistogram(window, i, xbin, ybin):
    #### kernel density estimation #########################################

    sigma_x = window['x_error'].values[i] / 3
    sigma_y = window['y_error'].values[i] / 3
    
    smooth_bins = np.zeros((height, width))
    
    if sigma_x > 0.4 and sigma_y > 0.4:
        for u in range(width):
            for v in range(height):
                smooth_bins[v][u] += kernel((u - xbin), sigma_x) \
                * kernel((v - ybin), sigma_y)
    else:
        smooth_bins[ybin][xbin] += 1
    
    '''
    if sigma_x > 0.4 and sigma_y > 0.4:
        var = multivariate_normal(mean=[xbin,ybin], cov=[[sigma_x**2,0],[0,sigma_y**2]])
        for u in range(width):
                for v in range(height):
                    smooth_bins[v][u] += var.pdf([u,v])
    
    '''
    #if i > 30 and i < 60:
    #    print(sigma_x, sigma_y, smooth_bins.sum())

    ######################################################################
    
    return smooth_bins

In [11]:
def updateHistograms(window, histos, norm_counts):
    
    for i in range(len(window)):
        # bin positions
        xbin = width/2  + floor(window['coordinates'].values[i][0] / 3)
        ybin = height/2 + floor(window['coordinates'].values[i][1] / 3)

        if xbin >= 0 and xbin < width and ybin >= 0 and ybin < height:
            
            smooth_bins = smoothHistogram(window, i, xbin, ybin)
            
            # update density histogram
            histos[window['sourceMac'].values[i]] += smooth_bins
            norm_counts[window['sourceMac'].values[i]] += 1
            
    return histos, norm_counts

In [31]:
def normalizeHistograms(histos, norm_counts):
    # total density histogram per period
    total_dens_histo = np.zeros((height, width))
    print(len(histos))
    
    for mac in histos.keys():
        if norm_counts[mac] > 0:
            total_dens_histo += histos[mac] / norm_counts[mac]
    return total_dens_histo

In [32]:
def runDataAnalysis():
    
    for k in range(periods):
        window = movingWindow(k)
        histos, norm_counts = createHistograms(window)
        histos, norm_counts = updateHistograms(window, histos, norm_counts)
        total_dens_histo = normalizeHistograms(histos, norm_counts)
        
        np.savetxt('output/test_dens_histo_%d.csv' %  k, total_dens_histo, delimiter=',')
        #print('Time window:', k)
       

In [33]:
from math import floor, ceil, sqrt, pi, exp
import numpy as np
from scipy.stats import multivariate_normal

# size of binned region (number of bins 3x3m)
width = 80; height = 60
# numbers of time intervals
periods = 60
timestep = 30000
interval = 120000

runDataAnalysis()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


493
560
594
672
731
743
743
721
728
756
770
787
786
800
750
722
700
670
720
772
773
802
820
807
817
826
820
817
808
810
788
783
779
722
742
708
678
660
606
604
591
604
587
521
492
467
464
524
514
523
522
520
508
503
514
492
476
476
515
507


In [19]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(16,10))

#col = ['r', 'y', 'c', 'k', 'c','r'] * height * width
col = ['w','r','w','w','w','w'] * height * width
# colors = np.random.choice(col, height*width)

for k in range(periods):
    
    ax = fig.add_subplot(111, projection='3d')

    x_data, y_data = np.meshgrid( np.arange(width),
                                  np.arange(height)*(-1) )

    x_data = x_data.flatten()
    y_data = y_data.flatten()

    z_data = np.loadtxt('output/test_dens_histo_%d.csv' % k, delimiter=',').flatten()
    #z_data = total_dens_histos[k].flatten()
    ax.set_zlim3d(0, 0.001)
    ax.bar3d( x_data,
              y_data,
              np.zeros(len(z_data)),
              1, 1, z_data, color=col) # 
    if k < 10:
        number = '000' + str(k)
    elif k > 9:
        number = '00' + str(k)
    elif k > 99:
        number = '0' + str(k)
    plt.savefig('output/smoothed-unity-histograms-2-%s.png' % number)

#plt.show()